In [2]:
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os
import requests
import json

import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from io import BytesIO

import time
import json
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm

### Accessing the wiki Art api

In [3]:
# Load our API access key
access_key = os.getenv('WIKIART_ACCESS_KEY')
secret_code = os.getenv('WIKIART_SECRET_KEY')

In [4]:
login_url = "https://www.wikiart.org/en/Api/2/login"

In [5]:
# Examine the most viewed artists list

base_url = "https://www.wikiart.org/en/api/2/MostViewedPaintings"

try:
    response = requests.get(base_url)
    response.raise_for_status()
    
    result = response.json()
    
    most_visited_df = pd.DataFrame(result.get('data', []))
    
    print("Download complete")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Download complete


In [6]:
login_url = "https://www.wikiart.org/en/Api/2/login"
painting_details_url = "https://www.wikiart.org/en/api/2/Painting"

# Painting details for "The Starry Night"
painting_id = "5772716cedc2cb3880c1907f" 

try:
    # Step 1: Create session
    login_params = {
        "accessCode": access_key,
        "secretCode": secret_code
    }
    login_response = requests.get(login_url, params=login_params)
    login_response.raise_for_status()
    
    login_data = login_response.json()
    session_key = login_data.get('SessionKey')
    
    if not session_key:
        raise Exception(f"Failed to obtain session key. Response: {login_data}")

    print(f"Successfully obtained session key")

    # Step 2: Retrieve painting details
    painting_params = {
        "id": painting_id,
        "authSessionKey": session_key  # Pass the session key
    }

    response = requests.get(painting_details_url, params=painting_params)

    if response.status_code == 200:
        painting_data = response.json()
        starry_night_df = pd.DataFrame([painting_data])  # Convert painting data to DataFrame
        print("Download complete")
        display(starry_night_df.head()) 
    elif response.status_code == 404:
        print(f"Painting not found for ID: {painting_id}")
    else:
        print(f"Failed to retrieve painting details for ID: {painting_id}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")
    if hasattr(e, 'response'):
        print(f"Response status code: {e.response.status_code}")
        print(f"Response content: {e.response.text}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully obtained session key
Download complete


,id,title,url,artistUrl,artistName,artistId,completitionYear,dictionaries,location,period,...,media,sizeX,sizeY,diameter,galleries,tags,description,width,image,height
0,5772716cedc2cb3880c1907f,The Starry Night,the-starry-night-1889,vincent-van-gogh,van Gogh Vincent,57726d82edc2cb3880b486a0,1889,"[57726b52edc2cb3880ad77e0, 57726b4eedc2cb3880a...",Saint-rémy-de-provenceFrance,None,...,"[oil, canvas]",92.1,73.7,None,"[Museum of Modern Art (MoMA), New York City, N...","[houses-and-buildings, twilight-and-night, Arl...",Van Gogh's night sky is a field of roiling ene...,750,https://uploads3.wikiart.org/00475/images/vinc...,598


In [7]:
painting_data

{'id': '5772716cedc2cb3880c1907f',
 'title': 'The Starry Night',
 'url': 'the-starry-night-1889',
 'artistUrl': 'vincent-van-gogh',
 'artistName': 'van Gogh Vincent ',
 'artistId': '57726d82edc2cb3880b486a0',
 'completitionYear': 1889,
 'dictionaries': ['57726b52edc2cb3880ad77e0',
  '57726b4eedc2cb3880ad6e38',
  '57726bcdedc2cb3880ae733e',
  '57726b4eedc2cb3880ad6f70'],
 'location': 'Saint-rémy-de-provenceFrance',
 'period': None,
 'serie': None,
 'genres': ['cloudscape'],
 'styles': ['Post-Impressionism'],
 'media': ['oil', 'canvas'],
 'sizeX': 92.1,
 'sizeY': 73.7,
 'diameter': None,
 'galleries': ['Museum of Modern Art (MoMA), New York City, NY, US'],
 'tags': ['houses-and-buildings',
  'twilight-and-night',
  'Arles',
  'stars-and-planets',
  'Tree',
  'Sky'],
 'description': 'Van Gogh\'s night sky is a field of roiling energy. Below the exploding stars, the village is a place of quiet order. Connecting earth and sky is the flamelike cypress, a tree traditionally associated with gr

In [8]:
starry_night_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1 non-null      object 
 1   title             1 non-null      object 
 2   url               1 non-null      object 
 3   artistUrl         1 non-null      object 
 4   artistName        1 non-null      object 
 5   artistId          1 non-null      object 
 6   completitionYear  1 non-null      int64  
 7   dictionaries      1 non-null      object 
 8   location          1 non-null      object 
 9   period            0 non-null      object 
 10  serie             0 non-null      object 
 11  genres            1 non-null      object 
 12  styles            1 non-null      object 
 13  media             1 non-null      object 
 14  sizeX             1 non-null      float64
 15  sizeY             1 non-null      float64
 16  diameter          0 non-null      object 
 17  g

In [9]:
starry_night_df['styles']

0    [Post-Impressionism]
Name: styles, dtype: object

In [10]:


def download_image(image_url, save_path):
    try:
        # Send a GET request to the image URL
        response = requests.get(image_url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Open the image using PIL
        img = Image.open(BytesIO(response.content))

        # Save the image
        img.save(save_path)
        print(f"Image successfully downloaded and saved to {save_path}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading the image: {e}")
    except IOError as e:
        print(f"Error saving the image: {e}")

# Assuming starry_night_df is the DataFrame containing the painting data
if 'starry_night_df' in locals() and not starry_night_df.empty:
    # Get the image URL from the DataFrame
    image_url = starry_night_df['image'].iloc[0]
    
    # Create a directory to save the image
    save_dir = "downloaded_images_wikiart"
    os.makedirs(save_dir, exist_ok=True)
    
    # Generate a filename for the image
    filename = f"starry_night_{painting_id}.jpg"
    save_path = os.path.join(save_dir, filename)
    
    # Download and save the image
    download_image(image_url, save_path)
else:
    print("Painting data not available. Please ensure you've successfully retrieved the painting details first.")

# Print the image URL for verification
print(f"Image URL: {image_url if 'image_url' in locals() else 'Not available'}")

Image successfully downloaded and saved to downloaded_images_wikiart\starry_night_5772716cedc2cb3880c1907f.jpg
Image URL: https://uploads3.wikiart.org/00475/images/vincent-van-gogh/the-starry-night-1889.jpg!Large.jpg


In [11]:
base_url = "https://www.wikiart.org/en/api/2"

def get_session_key():
    login_url = f"{base_url}/login"
    login_params = {
        "accessCode": access_key,
        "secretCode": secret_code
    }
    response = requests.get(login_url, params=login_params)
    response.raise_for_status()
    return response.json().get('SessionKey')

def make_api_request(endpoint, params):
    session_key = get_session_key()
    params['authSessionKey'] = session_key
    url = f"{base_url}/{endpoint}"
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def explore_dictionaries_by_group(group):
    print(f"\nExploring Dictionaries by Group {group}")
    try:
        data = make_api_request("DictionariesByGroup", {"group": group})
        print(json.dumps(data, indent=2))
    except Exception as e:
        print(f"Error: {e}")

def explore_artists_by_dictionary(group, dict_url):
    print(f"\nExploring Artists by Dictionary - Group: {group}, DictUrl: {dict_url}")
    try:
        data = make_api_request("ArtistsByDictionary", {"group": group, "dictUrl": dict_url})
        print(json.dumps(data, indent=2))
    except Exception as e:
        print(f"Error: {e}")
def explore_painting_search(term):
    print(f"\nExploring Painting Search - Term: {term}")
    try:
        data = make_api_request("PaintingSearch", {"term": term})
        print(json.dumps(data, indent=2))
    except Exception as e:
        print(f"Error: {e}")

# Example usage
explore_dictionaries_by_group(1)  # Explore group 1
explore_dictionaries_by_group(2)  # Explore group 2




Exploring Dictionaries by Group 1
{
  "data": [
    {
      "id": "5d230437edc2c9fb74756177",
      "title": "1st Intermediate Period (2181\u20132055 BC)",
      "url": "1st-intermediate-period-2181-2055-bc",
      "group": 1
    },
    {
      "id": "5d23049cedc2c9fb74782eb2",
      "title": "2nd Intermediate Period (1650\u20131550 BC)",
      "url": "2nd-intermediate-period-1650-1550-bc",
      "group": 1
    },
    {
      "id": "5d23052fedc2c9fb747bf1d9",
      "title": "3rd Intermediate Period (1069\u2013664 BC)",
      "url": "3rd-intermediate-period-1069-664-bc",
      "group": 1
    },
    {
      "id": "57726a68edc2ca38801d5111",
      "title": "Abbasid Period (750\u20131258)",
      "url": "abbasid-period-750-1258",
      "group": 1
    },
    {
      "id": "57726a67edc2ca38801d4d81",
      "title": "Abstract Art",
      "url": "abstract-art",
      "group": 1
    },
    {
      "id": "57726a67edc2ca38801d4e69",
      "title": "Abstract Expressionism",
      "url": "abstract

In [12]:
# After exploring dictionaries, we'll use a sample dict_url for Artists by Dictionary
# You'll need to replace 'sample_dict_url' with an actual URL from the DictionariesByGroup response
explore_artists_by_dictionary(1, "baroque")

# Example painting search
explore_painting_search("impressionism")


Exploring Artists by Dictionary - Group: 1, DictUrl: baroque
{
  "data": [
    {
      "id": "62502c309e4363244cf85596",
      "artistName": "Michelangelo Cerquozzi",
      "url": "michelangelo-cerquozzi",
      "lastNameFirst": null,
      "birthDay": "/Date(-58948387200000)/",
      "deathDay": "/Date(-9782640000000)/",
      "birthDayAsString": "102",
      "deathDayAsString": "1660",
      "image": "https://uploads3.wikiart.org/00387/images//h0027-l06020228.jpg!Portrait.jpg",
      "wikipediaUrl": null,
      "dictionaries": [
        "57726a66edc2ca38801d4cd1",
        "57726b4fedc2cb3880ad71d0"
      ],
      "periods": [],
      "series": [],
      "activeYearsStart": null,
      "activeYearsCompletion": null,
      "biography": "",
      "gender": "male",
      "originalArtistName": "",
      "relatedArtists": []
    },
    {
      "id": "625874c99e436338ccb9f54f",
      "artistName": "Simone Pignoni",
      "url": "simone-pignoni",
      "lastNameFirst": null,
      "birthDay

In [13]:
top_styles = [
    "Impressionism",
    "Realism",
    "Romanticism",
    "Expressionism",
    "Post-Impressionism",
    "Baroque",
    "Art Nouveau (Modern)",
    "Surrealism",
    "Symbolism",
    "Abstract Expressionism",
    "Neoclassicism",
    "Naïve Art (Primitivism)",
    "Rococo",
    "Cubism",
    "Northern Renaissance",
    "Academicism",
    "Pop Art",
    "Mannerism (Late Renaissance)",
    "Minimalism",
    "Conceptual Art",
    "Abstract Art",
    "Art Informel",
    "Early Renaissance",
    "Ukiyo-e",
    "Magic Realism",
    "Neo-Expressionism",
    "High Renaissance",
    "Contemporary Realism",
    "Color Field Painting",
    "Orientalism",
    "Lyrical Abstraction",
    "Fauvism",
    "Contemporary",
    "Op Art",
    "Neo-Impressionism",
    "Art-Deco",
]

In [14]:
# API configuration
BASE_URL = "https://www.wikiart.org/en/api/2"
REQUESTS_PER_2_5_SECONDS = 10
REQUESTS_PER_HOUR = 400

session_key = None

@sleep_and_retry
@limits(calls=REQUESTS_PER_2_5_SECONDS, period=2.5)
def call_api():
    pass

@sleep_and_retry
@limits(calls=REQUESTS_PER_HOUR, period=3600)
def hourly_limit():
    pass

def get_session_key():
    login_url = f"{BASE_URL}/login"
    login_params = {
        "accessCode": access_key,
        "secretCode": secret_code
    }
    try:
        response = requests.get(login_url, params=login_params)
        response.raise_for_status()
        return response.json().get('SessionKey')
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve session key: {e}")
        raise

def make_api_request(endpoint, params, max_retries=3, delay=5):
    global session_key
    if not session_key:
        session_key = get_session_key()

    for attempt in range(max_retries):
        try:
            call_api()
            hourly_limit()
            time.sleep(0.1)
            
            params['authSessionKey'] = session_key
            url = f"{BASE_URL}/{endpoint}"
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if 'limit exceeded' in str(e).lower():
                print("API rate limit exceeded. Waiting before retrying...")
                time.sleep(60)
            elif attempt < max_retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"Max retries reached. Unable to complete request to {endpoint}.")
                raise

def collect_style_artist_data(top_styles, start_group=600, end_group=799):
    matched_styles = {}
    artist_set = set()
    all_artists = []
    
    try:
        for group in range(start_group, end_group + 1):
            try:
                print(f"Processing group {group}")
                response = make_api_request("DictionariesByGroup", {"group": group})
                
                if not isinstance(response, dict) or 'data' not in response:
                    print(f"Unexpected response format for group {group}: {response}")
                    continue

                dictionaries = response['data']

                for dictionary in dictionaries:
                    style_title = dictionary.get('title')
                    if style_title in top_styles:
                        if style_title not in matched_styles:
                            matched_styles[style_title] = []
                        matched_styles[style_title].append({
                            'group': group,
                            'url': dictionary.get('url')
                        })

                print(f"Finished processing group {group}")  # Ensure this prints
            except Exception as e:
                print(f"Error processing group {group}: {e}")
                continue  # Continue to the next group

        print(f"Completed all groups up to {end_group}.")  # After loop exits

        for style, occurrences in matched_styles.items():
            for occurrence in occurrences:
                try:
                    response = make_api_request("ArtistsByDictionary", {
                        "group": occurrence['group'], 
                        "dictUrl": occurrence['url']
                    })

                    if not isinstance(response, dict) or 'data' not in response:
                        print(f"Unexpected response format for style {style}, group {occurrence['group']}: {response}")
                        continue

                    artists = response['data']
                
                    for artist in artists:
                        artist_id = artist.get('id')
                        if artist_id not in artist_set:
                            artist_set.add(artist_id)
                            all_artists.append({
                                'Style': style,
                                'Group': occurrence['group'],
                                'Artist ID': artist_id,
                                'Artist Name': artist.get('artistName'),
                                'Artist URL': artist.get('url')
                            })
                
                except Exception as e:
                    print(f"Error processing artists for style {style}, group {occurrence['group']}: {e}")
                    continue  # Continue to the next occurrence
    
    except KeyboardInterrupt:
        print("KeyboardInterrupt detected. Saving partial results...")

    finally:
        print("Reached finally block. Saving partial results...")
        df = pd.DataFrame(all_artists)
        df.to_csv('partial_results_4.csv', index=False)
        print(f"Partial results saved. Total records: {len(df)}")
        return df

# Usage

try:
    df = collect_style_artist_data(top_styles, start_group=600, end_group=799)
    display(df.head())
    print(f"Total records: {len(df)}")
except Exception as e:
    print(f"An error occurred: {e}")
    print("Check 'partial_results.csv' for the data collected so far.")


Processing group 600
Finished processing group 600
Processing group 601
Finished processing group 601
Processing group 602
Finished processing group 602
Processing group 603
Finished processing group 603
Processing group 604
Finished processing group 604
Processing group 605
Finished processing group 605
Processing group 606
Finished processing group 606
Processing group 607
Finished processing group 607
Processing group 608
Finished processing group 608
Processing group 609
Finished processing group 609
Processing group 610
Finished processing group 610
Processing group 611
Finished processing group 611
Processing group 612
Finished processing group 612
Processing group 613
Finished processing group 613
Processing group 614
Finished processing group 614
Processing group 615
Finished processing group 615
Processing group 616
Finished processing group 616
Processing group 617
Finished processing group 617
Processing group 618
Finished processing group 618
Processing group 619
Finished p

""


Total records: 0


Join batches and save

In [15]:
batch_1 = pd.read_csv("partial_results.csv")


In [16]:
batch_1['Style'].value_counts()

Style
Abstract Expressionism    60
Art Informel              60
Baroque                   60
Cubism                    60
Contemporary              59
Conceptual Art            58
Contemporary Realism      57
Expressionism             56
Early Renaissance         35
Name: count, dtype: int64

In [17]:
batch_1['Artist ID'].duplicated().any()

False

In [19]:
artist_ids = batch_1['Artist ID'].to_list()
artist_ids.remove('6671553f9e43634d88c0ba0c') # No pieces of artwork for this artist

In [21]:
import pandas as pd
import time
from urllib.parse import parse_qs, urlencode

def get_all_paintings_for_artist(artist_id):
    """Get all paintings for a single artist, handling pagination."""
    all_paintings = []
    pagination_token = None
    has_more = True
    
    while has_more:
        try:
            # Base parameters
            params = {'id': artist_id}
            
            # Handle pagination token separately to avoid double encoding
            if pagination_token:
                # Create URL with manually handled pagination token
                params = {
                    'id': artist_id,
                    'paginationToken': pagination_token,
                    'authSessionKey': session_key
                }
                
                # Construct URL manually to avoid double encoding
                url = f"{BASE_URL}/PaintingsByArtist"
                param_strings = []
                for key, value in params.items():
                    if key == 'paginationToken':
                        # Don't encode an already encoded token
                        param_strings.append(f"{key}={value}")
                    else:
                        # Encode other parameters normally
                        param_strings.append(f"{key}={requests.utils.quote(str(value))}")
                
                url = f"{url}?{'&'.join(param_strings)}"
                
                # Make request without params (they're in URL)
                response = requests.get(url)
                response.raise_for_status()
                response = response.json()
            else:
                # First page - use normal request
                response = make_api_request('PaintingsByArtist', params)
            
            if not response or 'data' not in response:
                print(f"No data returned for artist {artist_id}")
                break
                
            all_paintings.extend(response['data'])
            
            # Update pagination info
            has_more = response.get('hasMore', False)
            pagination_token = response.get('paginationToken')
            
            if has_more:
                time.sleep(0.5)  # Small delay between paginated requests
                
        except Exception as e:
            print(f"Error retrieving paintings for artist {artist_id}: {str(e)}")
            if all_paintings:  # Return what we got so far
                print(f"Returning {len(all_paintings)} paintings retrieved before error")
            break
            
    return all_paintings

def collect_all_paintings(artist_ids):
    """Collect paintings for all artists and return as DataFrame."""
    all_paintings = []
    
    for artist_id in artist_ids:
        try:
            paintings = get_all_paintings_for_artist(artist_id)
            print(f"Found {len(paintings)} paintings for artist {artist_id}")
            all_paintings.extend(paintings)
            
        except Exception as e:
            print(f"Failed to process artist {artist_id}: {str(e)}")
            continue
    
    # Convert to DataFrame
    if not all_paintings:
        print("No paintings were collected")
        return pd.DataFrame()
        
    df = pd.DataFrame(all_paintings)
    
    print(f"\nCollection complete!")
    print(f"Total paintings collected: {len(df)}")
    print(f"Total artists processed successfully: {df['artistId'].nunique()}")
    
    return df

# Execute the collection
paintings_df = collect_all_paintings(artist_ids)

Found 32 paintings for artist 636824e59e43633854e52d6f
Found 82 paintings for artist 617002ba9e43634a940b59e1
Found 100 paintings for artist 57726d9dedc2cb3880b4bc07
Found 26 paintings for artist 57726df4edc2cb3880b54838
Found 86 paintings for artist 57726d98edc2cb3880b4b44d
Found 29 paintings for artist 607afd46edc2c9a5b0cbce64
Found 22 paintings for artist 57726dd4edc2cb3880b51586
Found 16 paintings for artist 57726dbeedc2cb3880b4f25e
Found 20 paintings for artist 57726d9fedc2cb3880b4bf8a
Found 15 paintings for artist 57726dc7edc2cb3880b500ef
Found 19 paintings for artist 57726dcfedc2cb3880b50e2e
Found 14 paintings for artist 57726dfdedc2cb3880b55557
Found 34 paintings for artist 57726d9fedc2cb3880b4c002
Found 55 paintings for artist 5fb4bb7eedc2cbd4888a2aa3
Found 22 paintings for artist 57726dafedc2cb3880b4db50
Found 24 paintings for artist 57726dc4edc2cb3880b4fd3f
Found 51 paintings for artist 57726dadedc2cb3880b4d796
Found 65 paintings for artist 57726d9dedc2cb3880b4bc52
Found 170

In [22]:
paintings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22200 entries, 0 to 22199
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                22200 non-null  object 
 1   title             22200 non-null  object 
 2   url               22200 non-null  object 
 3   artistUrl         22200 non-null  object 
 4   artistName        22200 non-null  object 
 5   artistId          22200 non-null  object 
 6   completitionYear  17520 non-null  float64
 7   width             22200 non-null  int64  
 8   image             22200 non-null  object 
 9   height            22200 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 1.7+ MB


In [28]:
paintings_df = paintings_df[paintings_df['artistName'] != "Genis Remacha"]

In [29]:
paintings_df.head()

,id,title,url,artistUrl,artistName,artistId,completitionYear,width,image,height
32,61700aa89e43634a94520c9d,"Tintotela, cm 100x80 1974",tintotela-cm-100x80-1974-1974,ambrosio-paolo,ambrosio paolo,617002ba9e43634a940b59e1,1974.0,474,https://uploads4.wikiart.org/00363/images/ambr...,600
33,617037ee9e43634a94bc8587,70 41a Tintotela 1976,70-41a-tintotela-1976-1976,ambrosio-paolo,ambrosio paolo,617002ba9e43634a940b59e1,1976.0,750,https://uploads3.wikiart.org/00363/images/ambr...,591
34,617037ee9e43634a94bc858d,70 43,70-43-1976,ambrosio-paolo,ambrosio paolo,617002ba9e43634a940b59e1,1976.0,467,https://uploads2.wikiart.org/00363/images/ambr...,600
35,617037ee9e43634a94bc8591,70- 42,70-42-1976,ambrosio-paolo,ambrosio paolo,617002ba9e43634a940b59e1,1976.0,472,https://uploads2.wikiart.org/00363/images/ambr...,600
36,617037ee9e43634a94bc859b,70 -33,70-33-1977,ambrosio-paolo,ambrosio paolo,617002ba9e43634a940b59e1,1977.0,750,https://uploads4.wikiart.org/00363/images/ambr...,539


In [25]:
paintings_df['artistName'].value_counts()

artistName
Pablo Picasso                  1170
Peter Paul Rubens               616
Fernand Leger                   446
Ernst Ludwig Kirchner           391
MAUREEN QUIN                    268
                               ... 
Willem Danielsz van Tetrode       2
Abraham Bloemaert                 1
Arnold Schönberg                  1
Ferenc Pinter                     1
Benedetto Ghirlandaio             1
Name: count, Length: 494, dtype: int64

In [31]:
pablo_mask = paintings_df[paintings_df['artistName'] == 'Pablo Picasso']
pablo_mask

,id,title,url,artistUrl,artistName,artistId,completitionYear,width,image,height
13089,57727426edc2cb3880caf16e,The picador,the-picador-1890,pablo-picasso,Pablo Picasso,57726d84edc2cb3880b48c4d,1890.0,460,https://uploads1.wikiart.org/images/pablo-pica...,600
13090,57727407edc2cb3880ca6be4,House in the field,house-in-the-field-1893,pablo-picasso,Pablo Picasso,57726d84edc2cb3880b48c4d,1893.0,750,https://uploads8.wikiart.org/images/pablo-pica...,495
13091,57727416edc2cb3880cadd04,Plaster male torso,plaster-male-torso-1893,pablo-picasso,Pablo Picasso,57726d84edc2cb3880b48c4d,1893.0,379,https://uploads8.wikiart.org/images/pablo-pica...,600
13092,577273f9edc2cb3880ca5741,Academical study,academical-study,pablo-picasso,Pablo Picasso,57726d84edc2cb3880b48c4d,1895.0,438,https://uploads1.wikiart.org/images/pablo-pica...,600
13093,577273faedc2cb3880ca57c1,"Alicante, view of boats",alicante-view-of-boats,pablo-picasso,Pablo Picasso,57726d84edc2cb3880b48c4d,1895.0,750,https://uploads7.wikiart.org/images/pablo-pica...,472
...,...,...,...,...,...,...,...,...,...,...
14254,5772742bedc2cb3880caf94e,Untitled,untitled-1971-3,pablo-picasso,Pablo Picasso,57726d84edc2cb3880b48c4d,1971.0,483,https://uploads8.wikiart.org/images/pablo-pica...,381
14255,5772742dedc2cb3880cafd5e,Woman with braid,woman-with-braid-1971,pablo-picasso,Pablo Picasso,57726d84edc2cb3880b48c4d,1971.0,466,https://uploads6.wikiart.org/images/pablo-pica...,600
14256,57727406edc2cb3880ca6b24,Head of the man,head-of-the-man,pablo-picasso,Pablo Picasso,57726d84edc2cb3880b48c4d,1972.0,476,https://uploads0.wikiart.org/images/pablo-pica...,600
14257,5772819cedc2cb3880f5013d,Niko Pirosmani,niko-pirosmani-1972,pablo-picasso,Pablo Picasso,57726d84edc2cb3880b48c4d,1972.0,377,https://uploads7.wikiart.org/images/pablo-pica...,600


"Genis Remacha"

In [ ]:
wiki_art_final = pd.to_csv("wiki_art_final.csv", index=False)